In [35]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.1f}')
from cleaner import DataCleaner
import numpy as np
from datetime import datetime
from tools import (
    order_and_rename,
    import_datasets
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
movies = pl.read_csv('clean_datasets/movies.csv')
ratings = pl.read_csv('movies_datasets/tital_ratings.tsv', separator = "\t", ignore_errors = True)

joined = movies.join(
        ratings,
        left_on = "titre_id",
        right_on = "tconst"
    )

In [11]:
col_to_keep = [
    "titre_id",
    "titre_str",
    "titre_date_sortie",
    "titre_duree",
    "titre_genres",
    "person_id",
    "person_name",
    "person_birthdate",
    "person_job",
    "averageRating",
    "numVotes"
]

col_to_change = [
    "titre_id",
    "titre_str",
    "titre_date_sortie",
    "titre_duree",
    "titre_genres",
    "person_id",
    "person_name",
    "person_birthdate",
    "person_job",
    "rating_avg",
    "rating_votes"
]

df = order_and_rename(joined, col_to_keep, col_to_change)

In [21]:
path = "./clean_datasets/movies_ratings.csv"

df.write_csv("./clean_datasets/movies_ratings.csv")

In [25]:
rating = import_datasets(path, 'pandas', sep = ',')

2023-10-21 15:32:09 INFO     Pandas loaded ! Importing movies_ratings.csv...


In [28]:
cleaning = DataCleaner()
# d'abord je fix l'encodage et apres je clean les \N

clean_encode = cleaning.fix_values(rating, "fix_encode")
clean_n = cleaning.fix_values(clean_encode, "fix_n")
# maintenant je clean les porns ! et supprime les ligne ou nous n'avons pas de type de genre.
clean_porn = cleaning.clean_porn(clean_n, )

print(f"Cleaned {len(rating) - len(clean_porn)} rows")

rating = clean_porn

2023-10-21 15:33:06 INFO     Fixing encoding values...
2023-10-21 15:33:10 INFO     Fixing fucked up values...
2023-10-21 15:33:12 INFO     Cleaning porn movies...


Diif 116227


In [30]:
dates = ["<1900", ">1900", ">1930", ">1960", ">1990", ">2010"]

rating["cuts"] = pd.cut(
    rating["titre_date_sortie"],
    bins=[0, 1900, 1931, 1961, 1991, 2011, 2030],
    labels=dates
)

In [37]:
rating["rating_avg"].median()

6.2

In [36]:
rating["rating_avg"].describe()

count   2527386.0
mean          6.1
std           1.3
min           1.0
25%           5.3
50%           6.2
75%           7.0
max          10.0
Name: rating_avg, dtype: float64

In [ ]:
rating.loc[rating["rating_avg"].idxmax(), 'titre_str']

In [31]:
for date in dates[1:]:
    condi = (
        (rating["person_age"] != 0) &
        (rating["cuts"] == date)
    )

    print(f"Mediane for {date}", actor["person_age"][condi].median())

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,person_id,person_name,person_birthdate,person_job,rating_avg,rating_votes,cuts
0,tt0000009,Miss Jerry,1894.000,45,Romance,nm0063086,Blanche Bayliss,1878,actress,5.300,207,<1900
1,tt0000009,Miss Jerry,1894.000,45,Romance,nm0183823,William Courtenay,1875,actor,5.300,207,<1900
2,tt0000009,Miss Jerry,1894.000,45,Romance,nm1309758,Chauncey Depew,1834,actor,5.300,207,<1900
3,tt0000009,Miss Jerry,1894.000,45,Romance,nm0085156,Alexander Black,1859,director,5.300,207,<1900
4,tt0000147,The Corbett-Fitzsimmons Fight,1897.000,100,"Documentary,News,Sport",nm0179163,James J. Corbett,1866,self,5.300,483,<1900
...,...,...,...,...,...,...,...,...,...,...,...,...
2643608,tt9916730,6 Gunn,2017.000,116,Drama,nm10538612,Kiran Gawade,0,director,7.600,11,>2010
2643609,tt9916730,6 Gunn,2017.000,116,Drama,nm10538614,Ujjwala Gawde,0,producer,7.600,11,>2010
2643610,tt9916730,6 Gunn,2017.000,116,Drama,nm10538613,Abhishek Jathar,0,producer,7.600,11,>2010
2643611,tt9916730,6 Gunn,2017.000,116,Drama,nm1957275,Suresh Deshmane,0,cinematographer,7.600,11,>2010
